In [46]:
!pip install pandas==1.0.0

     |████████████████████████████████| 10.1MB 248kB/s eta 0:00:01
  Found existing installation: pandas 0.24.2
    Uninstalling pandas-0.24.2:
      Successfully uninstalled pandas-0.24.2


In [78]:
import pandas as pd
import numpy as np

  * https://data.humdata.org/dataset/afdb-socio-economic-database-1960-2019

In [79]:
df_ha = pd.read_csv('Data.csv')

In [80]:
df_ha.head()

,CountryName,Country,IndicatorName,Indicator,Units,Scale,Frequency,Date,Value
0,AFRICA,AFR,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,annual %,1,A,2000-01-01,9.133374
1,AFRICA,AFR,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,annual %,1,A,2001-01-01,8.535367
2,AFRICA,AFR,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,annual %,1,A,2002-01-01,7.238168
3,AFRICA,AFR,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,annual %,1,A,2003-01-01,7.217622
4,AFRICA,AFR,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,annual %,1,A,2004-01-01,7.373657


In [81]:
df_ha.CountryName.value_counts()

Tunisia                         31521
Tanzania                        31495
Senegal                         31408
Egypt                           31407
Morocco                         31403
                                ...  
ADF- only eligible countries     1068
ADF 13 Non Fragile                767
ADF 13 Fragile                    767
World                             337
Blend countries                   142
Name: CountryName, Length: 109, dtype: int64

In [82]:
cats = pd.Series(df_ha[df_ha["CountryName"] == "Tunisia"].IndicatorName.value_counts())

In [83]:
cats.head(100)[:50]

Errors & omissions (Cur, BoP, US $)                  85
Overall Balance (Cur, BoP, US$)                      85
Population aged 15-24, total                         61
Population Density (people per sq. km)               61
Population ages 15-49, female                        61
Population ages 65 and above, total                  61
Population ages 15-64, male                          61
Sex ratio (Males per 100 females)                    61
Age dependency ratio, +65 years                      61
Population aged 0-1, total                           61
Population ages 15-64, female                        61
Population ages 15-64, female (% of  total)          61
Population aged 0-4, male                            61
Population, female                                   61
Rate of Natural Increase (%)                         61
Population, Total                                    61
'Age dependency ratio, 0-14 years                    61
Life expectancy at birth, male (years)          

In [84]:
rel_cou = ['Central African Republic', 'Sudan',
       'Democratic Republic of Congo', 'Rwanda', 'Uganda', 'Morocco',
       'Republic of Congo', 'Kenya', 'Ivory Coast', 'Niger', 'eSwatini',
       'South Africa', 'Lesotho', 'Nigeria', 'Ethiopia', 'Angola',
       'Zambia', 'Mauritania', 'Burundi', 'Zimbabwe', 'Togo', 'Senegal',
       'Mozambique', 'Guinea-Bissau', 'Somalia', 'Namibia',
       'Equatorial Guinea', 'Madagascar', 'Eritrea', 'Cameroon',
       'Botswana', 'Guinea', 'Chad', 'Liberia', 'Algeria', 'Malawi',
       'Tunisia', 'Burkina Faso', 'Libya', 'Tanzania', 'Benin', 'Gabon',
       'Mali', 'Ghana', 'Gambia', 'Egypt', 'Sierra Leone', 'Djibouti',
       'South Sudan']

In [85]:
rel_cat = ["Life expectancy at birth, total (years)",
           "GDP (current US$)",
           "Death rate, crude (per 1,000 people)",
           "Population growth (annual %)",
           "Urban population (% of total)",
           "Central government, Fiscal Balance (Current US $)",
           "Population, Total"
          ]

In [203]:
df_cut = df_ha[(df_ha["CountryName"].isin(rel_cou)) & (df_ha["IndicatorName"].isin(rel_cat))]

In [204]:
df_cut["Date"] = pd.to_datetime(df_cut["Date"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [205]:
df_cut = df_cut[df_cut["Date"] >= pd.Timestamp(1997,1,1)]

In [206]:
df_cut = df_cut.drop(columns=['Indicator',])

In [207]:
df_cut.head()

,CountryName,Country,IndicatorName,Units,Scale,Frequency,Date,Value
52084,Cameroon,CMR,"Central government, Fiscal Balance (Current US $)",current US$,1,A,1997-01-01,-3.233041e+08
52085,Cameroon,CMR,"Central government, Fiscal Balance (Current US $)",current US$,1,A,1998-01-01,-3.604077e+08
52086,Cameroon,CMR,"Central government, Fiscal Balance (Current US $)",current US$,1,A,1999-01-01,-2.971021e+08
52087,Cameroon,CMR,"Central government, Fiscal Balance (Current US $)",current US$,1,A,2000-01-01,4.082743e+08
52088,Cameroon,CMR,"Central government, Fiscal Balance (Current US $)",current US$,1,A,2001-01-01,1.242385e+08


In [201]:
for cat in rel_cat:
    df_part = df_cut[df_cut["IndicatorName"] == cat]
    df_part[cat] = df_part["Value"]
    df_part.dropna(subset=[cat], inplace=True)
    #df_part.drop(columns=['Country', 'Units', 'Scale', 'Frequency', 'Value'])
    df_cut = df_cut.merge(how='inner', on=["CountryName","Date"], right=df_part)
    df_cut.rename(columns={"IndicatorName_y":"IndicatorName",
                           "Value_x":"Value"},inplace=True)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [202]:
df_cut

,Country_x_x_x_x_x_x_x,IndicatorName_x_x_x_x_x_x_x,Units_x_x_x_x_x_x_x,Scale_x_x_x_x_x_x_x,Frequency_x_x_x_x_x_x_x,Value,Country_y_x_x_x_x_x_x,IndicatorName_x_x_x_x_x_x,Units_y_x_x_x_x_x_x,Scale_y_x_x_x_x_x_x,...,Scale_y_y_y_y_y_y_y,Frequency_y_y_y_y_y_y_y,Value_y_y_y_y_y_y_y,"Life expectancy at birth, total (years)_y_y_y_y_y_y",GDP (current US$)_y_y_y_y_y,"Death rate, crude (per 1,000 people)_y_y_y_y",Population growth (annual %)_y_y_y,Urban population (% of total)_y_y,"Central government, Fiscal Balance (Current US $)_y","Population, Total"


In [149]:
pd.concat([df_part])

,CountryName,Country,IndicatorName,Units,Scale,Frequency,Date,Value
67851,Cameroon,CMR,"Population, Total",NaN,1,A,1997-01-01,14709961.0
67852,Cameroon,CMR,"Population, Total",NaN,1,A,1998-01-01,15108630.0
67853,Cameroon,CMR,"Population, Total",NaN,1,A,1999-01-01,15514249.0
67854,Cameroon,CMR,"Population, Total",NaN,1,A,2000-01-01,15927713.0
67855,Cameroon,CMR,"Population, Total",NaN,1,A,2001-01-01,16349364.0
...,...,...,...,...,...,...,...,...
1655958,Togo,TGO,"Population, Total",NaN,1,A,2016-01-01,7496833.0
1655959,Togo,TGO,"Population, Total",NaN,1,A,2017-01-01,7691915.0
1655960,Togo,TGO,"Population, Total",NaN,1,A,2018-01-01,7889759.0
1655961,Togo,TGO,"Population, Total",NaN,1,A,2019-01-01,8090336.0


In [124]:
df_cut = df_cut.reset_index()

In [113]:
def take_cols(row):
    row_d = row.to_dict()
    row_d[row["IndicatorName"]] = row["Value"] 
    return pd.Series(row_d)
    #insert(loc=int(row.name),column=row["IndicatorName"], value=row["Value"])

In [132]:
df

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            156, 157, 158, 159, 160, 161, 162, 163, 164, 165],
           dtype='int64', length=166)

In [122]:
df_social = pd.DataFrame(df_cut.apply(take_cols, axis=1))

In [123]:
df_social.set_index("Date")

,"Central government, Fiscal Balance (Current US $)",Country,CountryName,"Death rate, crude (per 1,000 people)",Frequency,GDP (current US$),IndicatorName,"Life expectancy at birth, total (years)",Population growth (annual %),"Population, Total",Scale,Units,Urban population (% of total),Value,index
Date,,,,,,,,,,,,,,,
1997-01-01,-3.233041e+08,CMR,Cameroon,NaN,A,NaN,"Central government, Fiscal Balance (Current US $)",NaN,NaN,NaN,1,current US$,NaN,-3.233041e+08,52084
1998-01-01,-3.604077e+08,CMR,Cameroon,NaN,A,NaN,"Central government, Fiscal Balance (Current US $)",NaN,NaN,NaN,1,current US$,NaN,-3.604077e+08,52085
1999-01-01,-2.971021e+08,CMR,Cameroon,NaN,A,NaN,"Central government, Fiscal Balance (Current US $)",NaN,NaN,NaN,1,current US$,NaN,-2.971021e+08,52086
2000-01-01,4.082743e+08,CMR,Cameroon,NaN,A,NaN,"Central government, Fiscal Balance (Current US $)",NaN,NaN,NaN,1,current US$,NaN,4.082743e+08,52087
2001-01-01,1.242385e+08,CMR,Cameroon,NaN,A,NaN,"Central government, Fiscal Balance (Current US $)",NaN,NaN,NaN,1,current US$,NaN,1.242385e+08,52088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-01-01,NaN,TGO,Togo,NaN,A,NaN,Urban population (% of total),NaN,NaN,NaN,1,% of total,39.666283,3.966628e+01,1660492
2017-01-01,NaN,TGO,Togo,NaN,A,NaN,Urban population (% of total),NaN,NaN,NaN,1,% of total,40.096725,4.009672e+01,1660493
2018-01-01,NaN,TGO,Togo,NaN,A,NaN,Urban population (% of total),NaN,NaN,NaN,1,% of total,40.524140,4.052414e+01,1660494


In [117]:
df_social = df_social[~np.isnan(df_social["Death rate, crude (per 1,000 people)"])]
df_social.shape

(1080, 16)

In [118]:
df_social = df_social[~np.isnan(df_social["Life expectancy at birth, total (years)"])]
df_social.shape

(0, 16)

df_social = df_social[~np.isnan(df_social["GDP (current US$)"])]

In [108]:
df_social = df_social[~np.isnan(df_social["Population growth (annual %)"])]

In [108]:
df_social = df_social[~np.isnan(df_social["Urban population (% of total)"])]

In [108]:
df_social = df_social[~np.isnan(df_social["Central government, Fiscal Balance (Current US $)"])]

In [108]:
df_social = df_social[~np.isnan(df_social["Population, Total"])]

In [109]:
df_social

,"Central government, Fiscal Balance (Current US $)",Country,CountryName,Date,"Death rate, crude (per 1,000 people)",Frequency,GDP (current US$),IndicatorName,"Life expectancy at birth, total (years)",Population growth (annual %),"Population, Total",Scale,Units,Urban population (% of total),Value,index


In [103]:
df_social.reset_index(inplace=True)

In [105]:
df_social

,level_0,"Central government, Fiscal Balance (Current US $)",Country,CountryName,Date,"Death rate, crude (per 1,000 people)",Frequency,GDP (current US$),"Life expectancy at birth, total (years)",Population growth (annual %),"Population, Total",Scale,Units,Urban population (% of total),Value,index


In [104]:
df_social.drop(columns=['IndicatorName','level_0'], inplace=True)

KeyError: "['IndicatorName'] not found in axis"

In [111]:
df_cut.rename(columns={"Life expectancy at birth, total (years)":"lifeexp",
                       "GDP (current US$)":"gdp",
                       "Death rate, crude (per 1,000 people)":"deathrate",
                       "Population growth (annual %)":"popgrowth",
                       "Urban population (% of total)":"urbpop",
                       "Central government, Fiscal Balance (Current US $)":"gov_fin",
                       "Population, Total":"pop_total"
                       },inplace=True)

In [208]:
df_cut.to_csv("social_data.csv")